# Dados Não Estruturados

```{admonition} Importante
:class: tip
Para execução dos códigos, é necessário instalar as seguintes bibliotecas: 

 - *!pip install -U scikit-learn*
 - *!pip install biopython*
 - *!pip install -U scikit-image*
 - *!pip install --no-binary :all: opencv-python*
 - *!pip install spaCy*
 - *!pip install -U spacy-lookups-data*
 - *!python -m spacy download pt_core_news_lg*
 - *!pip install nltk*
```

## Análise de Sequências Biológicas

In [1]:
# Importando Bibliotecas necessárias
import numpy as np
import pandas as pd
import collections
from Bio import SeqIO
from itertools import product


def janela_deslizante(seq, janela):
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+janela>seqlen else i+janela
        yield seq[i:j]
        if j==seqlen: break
    return


def k_possiveis(tamanho_k, tipo_seq):
    k_possiveis = [''.join(str(i) for i in x) for x in product(tipo_seq, repeat=tamanho_k)]
    dados_estruturados = pd.DataFrame(columns=range(len(k_possiveis)))
    dados_estruturados.columns = k_possiveis

    kmer = {}
    for k in k_possiveis:
        kmer[k] = 0
    kmer = collections.OrderedDict(sorted(kmer.items()))
    return dados_estruturados, kmer


def kmers(entrada, tamanho_k, tipo_seq):
    
    dados_estruturados, kmer = k_possiveis(tamanho_k, tipo_seq)
   
    for seq_record in SeqIO.parse(entrada, "fasta"):
        seq = seq_record.seq
        seq = seq.upper()   

        for subseq in janela_deslizante(seq, tamanho_k):
            try:
              kmer[subseq] = kmer[subseq] + 1
            except:
              pass 

        dados_estruturados = dados_estruturados.append(kmer, ignore_index=True)
        kmer = dict.fromkeys(kmer, 0)
    return dados_estruturados
    
    
dna = ['A', 'C', 'G', 'T']
rna = ['A', 'C', 'G', 'U']
protein = ['A', 'C', 'D', 'E', 'F', 
           'G', 'H', 'I', 'K', 'L', 
           'M', 'N', 'P', 'Q', 'R', 
           'S', 'T', 'V', 'W', 'Y']

pos_estruturado = kmers('positivo.fasta', 1, protein)
print(pos_estruturado)

neg_estruturado = kmers('negativo.fasta', 1, protein)
print(neg_estruturado)

ModuleNotFoundError: No module named 'numpy'

## Análise de Imagens

In [ ]:
# Importando bibliotecas para carregar e manipular os dados
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Diretorio base onde as imagens foram carregadas
base_dir = "./yalefaces"

# Leitura dos arquivos das imagens
image_files = [os.path.join(base_dir, file) for file in os.listdir(base_dir)]
images = [np.array(Image.open(img)) for img in image_files]

# Inicialização do detector de faces Haar Cascades através do arquivo obtido no repositório do OpenCV
faceDetectClassifier = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
faces_cropped = []

# Aplicação do detector de face e segmentando a imagem original para um tamanho de 150x150 contendo somente as faces
for img in images:
    facePoints = faceDetectClassifier.detectMultiScale(img)
    x, y = facePoints[0][:2]
    cropped = img[y: y + 150, x: x + 150]
    faces_cropped.append(cropped)

faces_cropped = np.array(faces_cropped)

fig, ax = plt.subplots(1, 2, figsize=(12,4))

ax[0].imshow(images[0], cmap='gray')
ax[1].imshow(faces_cropped[0], cmap='gray')

In [ ]:
from skimage.feature import local_binary_pattern

# Para evitar a divisão por um número próximo a zero
eps = 1e-7

feature_vector = []

for face in faces_cropped:

  # Obtenção da imagem com os padrões locais binários
  lbp = local_binary_pattern(image=face, 
                             P=8, R=1, 
                             method='uniform')

  # Representando e concatenando os padrões com um histograma
  n_bins = int(lbp.max() + 1)
  (hist, _) = np.histogram(lbp.ravel(),
                           bins=n_bins,
                           range=(0, n_bins))

  # Normalizando o histograma obtido
  hist = hist.astype("float")
  hist /= (hist.sum() + eps)

  feature_vector.append(hist)

feature_vector = np.array(feature_vector)

## Análise de Textos

In [ ]:
import pandas as pd
import spacy 
import nltk
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download('punkt')
pln = spacy.load('pt_core_news_lg')

def remove_acentos(df):
    df = df.str.replace('[àáâãäå]', 'a')
    df = df.str.replace('[èéêë]', 'e')
    df = df.str.replace('[ìíîï]', 'i')
    df = df.str.replace('[òóôõö]', 'o')
    df = df.str.replace('[ùúûü]', 'u')
    df = df.str.replace('[ç]', 'c')
    return df

def conversao_letras_minusculas(df):
    df = df.str.lower()
    return df

def remove_pontuacao(df):
    df = df.str.replace('[^\w\s]', '')
    return df

def remove_numeros(df):
    df = df.apply(lambda x: ' '.join([word for word in x.split() if not word.isdigit()]))
    return df

def remove_url(df):
    df = df.str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')
    return df

def remove_tag(df):
    df = df.str.replace(r'<[^<>]*>', '', regex=True)
    return df

def remove_palavras_frequentes(df, n_palavras):
    palavras = []
    textos = df.apply(nltk.word_tokenize)
    for texto in textos:
        for palavra in texto:
            palavras.append(palavra)
    freq = [x for x in nltk.FreqDist(palavras)]
    frequentes = freq[0:n_palavras]
    df = df.apply(lambda x: ' '.join([word for word in x.split() if word not in (frequentes)]))
    return df

def remove_palavras_raras(df, n_palavras):
    palavras = []
    textos = df.apply(nltk.word_tokenize)
    for texto in textos:
        for palavra in texto:
            palavras.append(palavra)
    freq = [x for x in nltk.FreqDist(palavras)]
    raras = freq[-n_palavras:]
    df = df.apply(lambda x: ' '.join([word for word in x.split() if word not in (raras)]))
    return df

def palavras_vazias(df):
    stopwords = pln.Defaults.stop_words
    df = df.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
    return df

def lematizacao(df):
    df = df.apply(lambda x: ' '.join([word.lemma_ for word in pln(x)]))
    return df

def saco_de_palavras(df):
    metodo = CountVectorizer(ngram_range=(1,1), stop_words=pln.Defaults.stop_words)
    X = metodo.fit_transform(df)
    df = pd.DataFrame(X.toarray(), columns=metodo.get_feature_names())
    return df

def tf_idf(df):
    metodo = TfidfVectorizer()
    X = metodo.fit_transform(df)
    df = pd.DataFrame(X.toarray(), columns=metodo.get_feature_names())
    return df


df = pd.read_csv('FACTCKBR.tsv', sep='\t')
df = df.rename(columns={'title': 'titulo'})

df['titulo'] = remove_acentos(df['titulo'])
df['titulo'] = conversao_letras_minusculas(df['titulo'])
df['titulo'] = remove_pontuacao(df['titulo'])
df['titulo'] = remove_numeros(df['titulo'])
df['titulo'] = remove_url(df['titulo'])
df['titulo'] = remove_tag(df['titulo'])
df['titulo'] = palavras_vazias(df['titulo'])
df['titulo'] = lematizacao(df['titulo'])
df['titulo'] = remove_palavras_frequentes(df['titulo'], 20)
df['titulo'] = remove_palavras_raras(df['titulo'], 20)
conjunto_final = tf_idf(df['titulo'])